In [19]:
from qiskit import BasicAer
from qiskit.aqua.operators import X, Z, I
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import aqua_globals

h = 0.5

H = (Z ^ Z ^ I ^ I ^ I) + \
    (I ^ Z ^ Z ^ I ^ I) + \
    (I ^ I ^ Z ^ Z ^ I) + \
    (I ^ I ^ I ^ Z ^ Z) + \
    (h * X ^ I ^ I ^ I ^ I) + \
    (h * I ^ X ^ I ^ I ^ I) + \
    (h * I ^ I ^ X ^ I ^ I) + \
    (h * I ^ I ^ I ^ X ^ I) + \
    (h * I ^ I ^ I ^ I ^ X) 

seed = 50
aqua_globals.random_seed = seed
qi = QuantumInstance(BasicAer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)

ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
slsqp = SLSQP(maxiter=1000)
vqe = VQE(operator=H, var_form=ansatz, optimizer=slsqp, quantum_instance=qi)
result = vqe.run()

import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result)

print('Energy: ', result.eigenvalue)

{   'cost_function_evals': 1971,
    'eigenstate': array([-0.01635262+6.09106577e-18j,  0.00944737-9.63391001e-19j,
        0.12033955+3.68572189e-17j, -0.03258666-2.13869620e-17j,
        0.0054853 +3.12593482e-17j, -0.00246734-8.25062491e-18j,
       -0.01383083-1.33593144e-17j,  0.00333819+4.35925368e-18j,
        0.12022947+2.02543901e-16j, -0.08213945-9.39724982e-17j,
       -0.91136662-1.61676339e-16j,  0.22627187+4.58721208e-17j,
       -0.0341934 -2.49327412e-18j,  0.01689713+8.06955031e-18j,
        0.11782525+1.29466294e-16j, -0.02629836-4.03318054e-17j,
        0.01148931+2.62504882e-17j, -0.00576898-6.95074346e-18j,
       -0.08189404-3.11641537e-17j,  0.02315388+9.10196643e-18j,
       -0.00757213+1.81484008e-16j,  0.0023889 -4.96699462e-17j,
        0.02159293-2.79471675e-18j, -0.00650032-6.11482326e-18j,
       -0.03055024-1.33248383e-16j,  0.01912473+4.40527950e-17j,
        0.22570367-1.58446758e-16j, -0.05950208+5.82583181e-17j,
        0.0068488 +7.86889073e-17j, -0.

In [20]:
from tenpy.networks.mps import MPS
from tenpy.models.tf_ising import TFIChain
from tenpy.algorithms import dmrg
import matplotlib.pyplot as plt

def DMRG_TFI(L, h, chi_max = 30, verbose=0, bc = 'finite', perturbation=False, orthogonal=[]):
    
    '''DMRG in the TFI model in a finite 1D chain'''

    model_params = dict(L=L, J=1., g=h, bc_MPS=bc, conserve=None, verbose=verbose)
    M = TFIChain(model_params)
    psi = MPS.from_product_state(M.lat.mps_sites(), ["up"] * M.lat.N_sites, bc=M.lat.bc_MPS)
   
    dmrg_params = {                                                                                             
        'mixer': True,                                                                                          
        'trunc_params': {                                                                                       
        'chi_max': chi_max,                                                                                                                                                                    
        },                                                                                                      
        'max_E_err': 1.e-16,                                                                                    
        'verbose': 0,
        'orthogonal_to': orthogonal}
    
    info = dmrg.run(psi, M, dmrg_params)
    
    return info['E'], psi

L = 5
h = 0.5

E, psi = DMRG_TFI(L, h, chi_max = 100)
print('Energy: ', E)

Energy:  -4.469490344005437
